In [1]:
from dask.distributed import Client, LocalCluster
# from dask_jobqueue import SLURMCluster

# # cluster = LocalCluster()
# cluster = SLURMCluster(cores=24, name='test', walltime='00:30:00',
#                        job_extra=['--constraint=HSW24','--exclusive','--nodes=1'], 
#                        memory='118GB', interface='ib0'
#                       ) 
# cluster.scale(120)
# # # cluster.adapt(minimum=1, maximum=48)

# client = Client(cluster)
client = Client()
client

/scratch/cnt0024/hmg2840/tuchida/condapack/lib/python3.7/site-packages/distributed/dashboard/core.py:72: UserWarning: 
Failed to start diagnostics server on port 8787. [Errno 13] Permission denied
  warnings.warn("\n" + msg)


Client Scheduler: tcp://127.0.0.1:42199 Dashboard: http://127.0.0.1:38926/status,Cluster Workers: 8 Cores: 48 Memory: 134.91 GB


In [2]:
!squeue -u tuchida

import time
nb_workers = 0
while True:
    nb_workers = len(client.scheduler_info()["workers"])
    if nb_workers >= 2:
        break
    time.sleep(1)
print(nb_workers)

             JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON)
8


In [3]:
import xarray as xr
import numpy as np
import xscale
import gsw
import os.path as op
from xhistogram.xarray import histogram as xhist

from matplotlib.gridspec import GridSpec
import matplotlib.pyplot as plt
%matplotlib inline

In [4]:
ddir = '/store/CT1/hmg2840/lbrodeau/eNATL60/eNATL60-BLBT02-S/'
xtra = '/store/CT1/hmg2840/lbrodeau/eNATL60/eNATL60-BLBT02X-S'
scratch = '/scratch/cnt0024/hmg2840/tuchida/temp'

In [5]:
g = 9.81
Ce = 0.01
ys,ye = (2000,2500)
xs,xe = (2300,3100)
zchunk = 4

dsmask = xr.open_dataset(op.join(ddir,'../eNATL60-I/mesh_mask_eNATL60_3.6.nc'), chunks={'z':zchunk})
dsmask

<xarray.Dataset>
Dimensions:       (t: 1, x: 8354, y: 4729, z: 300)
Dimensions without coordinates: t, x, y, z
Data variables:
    nav_lon       (y, x) float32 dask.array<chunksize=(4729, 8354), meta=np.ndarray>
    nav_lat       (y, x) float32 dask.array<chunksize=(4729, 8354), meta=np.ndarray>
    nav_lev       (z) float32 dask.array<chunksize=(4,), meta=np.ndarray>
    time_counter  (t) float64 dask.array<chunksize=(1,), meta=np.ndarray>
    tmask         (t, z, y, x) int8 dask.array<chunksize=(1, 4, 4729, 8354), meta=np.ndarray>
    umask         (t, z, y, x) int8 dask.array<chunksize=(1, 4, 4729, 8354), meta=np.ndarray>
    vmask         (t, z, y, x) int8 dask.array<chunksize=(1, 4, 4729, 8354), meta=np.ndarray>
    fmask         (t, z, y, x) int8 dask.array<chunksize=(1, 4, 4729, 8354), meta=np.ndarray>
    tmaskutil     (t, y, x) int8 dask.array<chunksize=(1, 4729, 8354), meta=np.ndarray>
    umaskutil     (t, y, x) int8 dask.array<chunksize=(1, 4729, 8354), meta=np.ndarray>
    vmaskutil     (t, y, x) int8 dask.array<chunksize=(1, 4729, 8354), meta=np.ndarray>
    fmaskutil     (t, y, x) int8 dask.array<chunksize=(1, 4729, 8354), meta=np.ndarray>
    glamt         (t, y, x) float32 dask.array<chunksize=(1, 4729, 8354), meta=np.ndarray>
    glamu         (t, y, x) float32 dask.array<chunksize=(1, 4729, 8354), meta=np.ndarray>
    glamv         (t, y, x) float32 dask.array<chunksize=(1, 4729, 8354), meta=np.ndarray>
    glamf         (t, y, x) float32 dask.array<chunksize=(1, 4729, 8354), meta=np.ndarray>
    gphit         (t, y, x) float32 dask.array<chunksize=(1, 4729, 8354), meta=np.ndarray>
    gphiu         (t, y, x) float32 dask.array<chunksize=(1, 4729, 8354), meta=np.ndarray>
    gphiv         (t, y, x) float32 dask.array<chunksize=(1, 4729, 8354), meta=np.ndarray>
    gphif         (t, y, x) float32 dask.array<chunksize=(1, 4729, 8354), meta=np.ndarray>
    e1t           (t, y, x) float64 dask.array<chunksize=(1, 4729, 8354), meta=np.ndarray>
    e1u           (t, y, x) float64 dask.array<chunksize=(1, 4729, 8354), meta=np.ndarray>
    e1v           (t, y, x) float64 dask.array<chunksize=(1, 4729, 8354), meta=np.ndarray>
    e1f           (t, y, x) float64 dask.array<chunksize=(1, 4729, 8354), meta=np.ndarray>
    e2t           (t, y, x) float64 dask.array<chunksize=(1, 4729, 8354), meta=np.ndarray>
    e2u           (t, y, x) float64 dask.array<chunksize=(1, 4729, 8354), meta=np.ndarray>
    e2v           (t, y, x) float64 dask.array<chunksize=(1, 4729, 8354), meta=np.ndarray>
    e2f           (t, y, x) float64 dask.array<chunksize=(1, 4729, 8354), meta=np.ndarray>
    ff            (t, y, x) float64 dask.array<chunksize=(1, 4729, 8354), meta=np.ndarray>
    mbathy        (t, y, x) int16 dask.array<chunksize=(1, 4729, 8354), meta=np.ndarray>
    misf          (t, y, x) int16 dask.array<chunksize=(1, 4729, 8354), meta=np.ndarray>
    isfdraft      (t, y, x) float32 dask.array<chunksize=(1, 4729, 8354), meta=np.ndarray>
    e3t_0         (t, z, y, x) float64 dask.array<chunksize=(1, 4, 4729, 8354), meta=np.ndarray>
    e3u_0         (t, z, y, x) float64 dask.array<chunksize=(1, 4, 4729, 8354), meta=np.ndarray>
    e3v_0         (t, z, y, x) float64 dask.array<chunksize=(1, 4, 4729, 8354), meta=np.ndarray>
    e3w_0         (t, z, y, x) float64 dask.array<chunksize=(1, 4, 4729, 8354), meta=np.ndarray>
    gdept_0       (t, z, y, x) float32 dask.array<chunksize=(1, 4, 4729, 8354), meta=np.ndarray>
    gdepu         (t, z, y, x) float32 dask.array<chunksize=(1, 4, 4729, 8354), meta=np.ndarray>
    gdepv         (t, z, y, x) float32 dask.array<chunksize=(1, 4, 4729, 8354), meta=np.ndarray>
    gdepw_0       (t, z, y, x) float32 dask.array<chunksize=(1, 4, 4729, 8354), meta=np.ndarray>
    gdept_1d      (t, z) float64 dask.array<chunksize=(1, 4), meta=np.ndarray>
    gdepw_1d      (t, z) float64 dask.array<chunksize=(1, 4), meta=np.ndarray>
    e3t_1d        (t, z) float64 dask.array<chunksize=(1, 4), meta=np.ndarray

In [6]:
At = (dsmask.e1t * dsmask.e2t)[0].sel(y=slice(ys,ye),x=slice(xs,xe))
Au = (dsmask.e1u * dsmask.e2u)[0].sel(y=slice(ys,ye),x=slice(xs,xe))
Av = (dsmask.e1v * dsmask.e2v)[0].sel(y=slice(ys,ye),x=slice(xs,xe))
# e1u = dsmask.e1u[0].sel(y=slice(ys,ye),x=slice(xs,xe))
# e2u = dsmask.e2u[0].sel(y=slice(ys,ye),x=slice(xs,xe))
# e1v = dsmask.e1v[0].sel(y=slice(ys,ye),x=slice(xs,xe))
# e2v = dsmask.e2v[0].sel(y=slice(ys,ye),x=slice(xs,xe))
e2t = dsmask.e2t[0].sel(y=slice(ys,ye),x=slice(xs,xe))
e1t = dsmask.e1t[0].sel(y=slice(ys,ye),x=slice(xs,xe))
e3t = dsmask.e3t_1d[0]
e3w = dsmask.e3w_1d[0]

In [7]:
cori = gsw.f(dsmask.nav_lat.sel(y=slice(ys,ye),x=slice(xs,xe)))
fb = xr.apply_ufunc(gsw.f,(dsmask.nav_lat.sel(y=slice(ys,ye),x=slice(xs,xe))*At).coarsen({'y':5,'x':5}, boundary='trim').sum()
                           / At.coarsen({'y':5,'x':5}, boundary='trim').sum(),
                    dask='parallelized', output_dtypes=[float,]
                   )
e1tb = e1t.coarsen({'y':5,'x':5},boundary='trim').sum()
e2tb = e2t.coarsen({'y':5,'x':5},boundary='trim').sum()
Aub = Au.coarsen({'y':5,'x':5}, boundary='trim').sum()
Avb = Av.coarsen({'y':5,'x':5}, boundary='trim').sum()
# Atb = At.coarsen({'y':5,'x':5}, boundary='trim').sum()

In [ ]:
# def filt(da, area=None):
    
# #     if area is not None:
#         win_2D = (da * area).window
#     else:
#         win_2D = da.window
    
#     win_2D.set(window='hanning', cutoff=20, dim=['y','x'], n=[30,30])
#     bw = win_2D.boundary_weights(drop_dims=[])
#     da_LS = win_2D.convolve(weights=bw)
    
#     if area is not None:
#         area_win = area.window
#         area_win.set(window='hanning', cutoff=20, dim=['y','x'], n=[30,30])
#         bw = area_win.boundary_weights(drop_dims=[])
#         da_LS *= area_win.convolve(weights=bw)**-1
    
#     da_SS = da - da_LS
    
#     return da_SS

In [8]:
xr.open_dataset(op.join(ddir,'00658801-00669600/eNATL60-BLBT02_1h_20091102_20091106_gridW_20091102-20091102.nc'),
               chunks=({'time_counter':1,'depthw':1})).sel(depthw=slice(None,510)).vovecrtz[0,0].data.nbytes*1e-6

158.024264

In [9]:
dsW = xr.open_mfdataset(op.join(ddir,'00658801-00669600/eNATL60-BLBT02_1h_20091102_20091106_gridW_200911*.nc'),
                        chunks={'time_counter':1,'depthw':1}, combine='by_coords'
                       ).sel(depthw=slice(None,510))
dsT = xr.open_mfdataset(op.join(ddir,'00658801-00669600/eNATL60-BLBT02_1h_20091102_20091106_gridT_200911*.nc'),
                        chunks={'time_counter':1,'deptht':1}, combine='by_coords'
                       ).sel(deptht=slice(None,510))
dsS = xr.open_mfdataset(op.join(ddir,'00658801-00669600/eNATL60-BLBT02_1h_20091102_20091106_gridS_200911*.nc'),
                        chunks={'time_counter':1,'deptht':1}, combine='by_coords'
                       ).sel(deptht=slice(None,510))
dsW

<xarray.Dataset>
Dimensions:               (axis_nbounds: 2, depthw: 74, time_counter: 120, x: 8354, y: 4729)
Coordinates:
    nav_lat               (y, x) float32 dask.array<chunksize=(4729, 8354), meta=np.ndarray>
    nav_lon               (y, x) float32 dask.array<chunksize=(4729, 8354), meta=np.ndarray>
  * depthw                (depthw) float32 0.0 1.0000261 ... 487.90018 500.50638
    time_centered         (time_counter) datetime64[ns] dask.array<chunksize=(1,), meta=np.ndarray>
  * time_counter          (time_counter) datetime64[ns] 2009-11-02T00:30:00 ... 2009-11-06T23:30:00
Dimensions without coordinates: axis_nbounds, x, y
Data variables:
    depthw_bounds         (time_counter, depthw, axis_nbounds) float32 dask.array<chunksize=(24, 1, 2), meta=np.ndarray>
    time_centered_bounds  (time_counter, axis_nbounds) datetime64[ns] dask.array<chunksize=(1, 2), meta=np.ndarray>
    time_counter_bounds   (time_counter, axis_nbounds) datetime64[ns] dask.array<chunksize=(1, 2), meta=np.ndarray>
    vovecrtz              (time_counter, depthw, y, x) float32 dask.array<chunksize=(1, 1, 4729, 8354), meta=np.ndarray>
Attributes:
    name:         /scratch/tmp/5091688/eNATL60-BLBT02_1h_20091102_20091106_gridW
    description:  ocean W grid variables
    title:        ocean W grid variables
    Conventions:  CF-1.6
    timeStamp:    2019-Mar-09 11:57:06 GMT
    uuid:         b541dd09-f592-4c7f-a4fa-0e01f9b6bcfa
    ibegin:       0
    ni:           8354
    jbegin:       0
    nj:           10
    file_name:    eNATL60-BLBT02_1h_20091102_20091106_gridW_20091102-20091102.nc
    TimeStamp:    10/03/2019 00:59:17 +0100

In [10]:
maskT = dsmask.tmask[0].sel(y=slice(ys,ye),x=slice(xs,xe)).isel(z=slice(None,len(dsW.depthw)))
    
CT = dsT.votemper.sel(y=slice(ys,ye),x=slice(xs,xe)
                     ).where(xr.DataArray(maskT.data, dims=['deptht','y','x']).chunk({'deptht':1})!=0.
                            )
SA = dsS.vosaline.sel(y=slice(ys,ye),x=slice(xs,xe)
                     ).where(xr.DataArray(maskT.data, dims=['deptht','y','x']).chunk({'deptht':1})!=0.
                            )
w = dsW.vovecrtz.sel(y=slice(ys,ye),x=slice(xs,xe)
                    ).where(xr.DataArray(maskT.data, dims=['depthw','y','x']).chunk({'deptht':1})!=0.
                           )

buoy = -g*xr.apply_ufunc(gsw.sigma0, SA, CT, 
                         dask='parallelized', output_dtypes=[float,]
                        )*1e-3

ws = filt(w.chunk({'depthw':1}), area=At)
bs = filt(buoy.chunk({'deptht':1}), area=At)

wsbs = (ws.isel(depthw=slice(1,None)) 
        * bs.chunk({'deptht':-1}).interp({'deptht':ws.depthw.isel(depthw=slice(1,None)).data},
                                         method='linear').data
       )
wsbs

<xarray.DataArray (time_counter: 120, depthw: 73, y: 500, x: 800)>
dask.array<mul, shape=(120, 73, 500, 800), dtype=float64, chunksize=(1, 1, 500, 800), chunktype=numpy.ndarray>
Coordinates:
  * depthw         (depthw) float64 1.0 2.157 3.471 4.943 ... 475.4 487.9 500.5
    nav_lat        (y, x) float32 dask.array<chunksize=(500, 800), meta=np.ndarray>
    nav_lon        (y, x) float32 dask.array<chunksize=(500, 800), meta=np.ndarray>
    time_centered  (time_counter) datetime64[ns] dask.array<chunksize=(1,), meta=np.ndarray>
  * time_counter   (time_counter) datetime64[ns] 2009-11-02T00:30:00 ... 2009-11-06T23:30:00
Dimensions without coordinates: y, x

In [ ]:
wsbs[:,15].mean('time_counter').plot(figsize=(12,7), vmin=-1e-7, vmax=1e-7, cmap='RdBu_r')

In [ ]:
wsbs_b = ((wsbs*At).coarsen({'y':5,'x':5},boundary='trim').sum() 
          * At.coarsen({'y':5,'x':5},boundary='trim').sum()**-1
         )
s_b = ((SA*At).coarsen({'y':5,'x':5}, boundary='trim').sum()
       / At.coarsen({'y':5,'x':5}, boundary='trim').sum()
      )
t_b = ((CT*At).coarsen({'y':5,'x':5}, boundary='trim').sum()
       / At.coarsen({'y':5,'x':5}, boundary='trim').sum()
      )

z10 = 6
sig0_b = xr.apply_ufunc(gsw.sigma0, s_b, t_b, 
                        dask='parallelized', output_dtypes=[float,]
                       ).persist()
b_b = -g*sig0_b*1e-3

nMLD_b = z10 + np.abs((sig0_b.isel(deptht=slice(z10,None))
                       - sig0_b.isel(deptht=z10)
                      ) - 0.03).argmin(dim='deptht')

In [13]:
import dask.array as dsar
xr.concat([xr.DataArray(np.random.random((2,4)), dims=['time','x']).chunk({'time':1}),
           xr.DataArray(np.random.random((3,4)), dims=['time','x']).chunk({'time':1})], 'time')

<xarray.DataArray (time: 5, x: 4)>
dask.array<concatenate, shape=(5, 4), dtype=float64, chunksize=(1, 4), chunktype=numpy.ndarray>
Dimensions without coordinates: time, x

In [8]:
# dirs = np.arange(669601,756000,43200,dtype=int)
days = np.concatenate((np.arange(26,32,dtype=int), np.arange(1,5,dtype=int))).reshape((2,5))
# days = np.arange(5,20, dtype=int).reshape((3,5))
dirs = np.repeat(np.array([842401]),days.shape[0]+1)
print(days,dirs)

[[11 12 13 14 15]
 [16 17 18 19 20]
 [21 22 23 24 25]
 [26 27 28 29 30]
 [ 1  2  3  4  5]] [1004401 1004401 1004401 1004401 1004401 1004401]


$$\langle w'C'\rangle \sim \langle w'b'\rangle \frac{\partial\langle C\rangle}{\partial\langle b\rangle}$$
<!-- $$\ \ \ \ \ \ = \frac{{H_{\tt ML}}^2|\overline{\nabla_{\tt H}\langle b\rangle}^z|^2}{f} \mu \frac{\partial z}{\partial b}\frac{\partial\theta}{\partial z}$$
$$\ \ \ \ \ \ \simeq \frac{{H_{\tt ML}}^2|\overline{\nabla_{\tt H}\langle b\rangle}^z|^2}{f} \mu \frac{1}{N^2}\frac{\partial\theta}{\partial z}$$
$$\ \ \ \ \ \ \ \ \sim \frac{{H_{\tt ML}}^2|\overline{\nabla_{\tt H}\langle b\rangle}^z|^2}{f} \frac{1}{\overline{N^2}^z}\frac{\Delta\theta}{H_{\tt ML}} \mu(z)$$ -->
$$\ \ \ \ \ \ \ \ \ \ \simeq \frac{{H_{\tt ML}}^2|\overline{\nabla_{\tt H}\langle b\rangle}^z|^2}{f} \mu \Big(\frac{\partial{\bf x}}{\partial\langle b\rangle}\frac{\partial\langle C\rangle}{\partial{\bf x}}\Big)$$

In [10]:
month = 4
year = 2010
for k in range(0,len(dirs)-1):
    for i in days[k]:
        j = month
        l = j
        m = month+1
#         if k > 0:
#             j = m
        if i < days[0,0]:
            l = m
        dsW = xr.open_dataset(op.join(xtra,'%08d-%08d/eNATL60-BLBT02X_1h_%4d%02d%02d_%4d%02d%02d_gridW_%4d%02d%02d-%4d%02d%02d.nc' 
                                      % (dirs[k],int(dirs[k]+10800*days.shape[0]-1),
                                         year,j,days[0,0],year,m,days[-1,-1],
                                         year,l,i,year,l,i)),
                              chunks={'time_counter':1,'depthw':zchunk}
                             ).sel(depthw=slice(None,510))
        dsT = xr.open_dataset(op.join(xtra,'%08d-%08d/eNATL60-BLBT02X_1h_%4d%02d%02d_%4d%02d%02d_gridT_%4d%02d%02d-%4d%02d%02d.nc' 
                                      % (dirs[k],int(dirs[k]+10800*days.shape[0]-1),
                                         year,j,days[0,0],year,m,days[-1,-1],
                                         year,l,i,year,l,i)),
                              chunks={'time_counter':1,'deptht':zchunk}
                             ).sel(deptht=slice(None,510))
        dsS = xr.open_dataset(op.join(xtra,'%08d-%08d/eNATL60-BLBT02X_1h_%4d%02d%02d_%4d%02d%02d_gridS_%4d%02d%02d-%4d%02d%02d.nc' 
                                      % (dirs[k],int(dirs[k]+10800*days.shape[0]-1),
                                         year,j,days[0,0],year,m,days[-1,-1],
                                         year,l,i,year,l,i)),
                              chunks={'time_counter':1,'deptht':zchunk}
                             ).sel(deptht=slice(None,510))

#         if i == days[0,0]:
        maskT = dsmask.tmask[0].sel(y=slice(ys,ye),x=slice(xs,xe)
                                    ).isel(z=slice(None,len(dsT.deptht)))

        CT = dsT.votemper[-1].sel(y=slice(ys,ye),x=slice(xs,xe)
                                 ).where(xr.DataArray(maskT.data, dims=['deptht','y','x']
                                                 ) != 0.)
        SA = dsS.vosaline[-1].sel(y=slice(ys,ye),x=slice(xs,xe)
                                 ).where(xr.DataArray(maskT.data, dims=['deptht','y','x']
                                                 ) != 0.)


#######################
        s_b = ((SA*At).coarsen({'y':5,'x':5}, boundary='trim').sum()
               / At.coarsen({'y':5,'x':5}, boundary='trim').sum()
              )
        t_b = ((CT*At).coarsen({'y':5,'x':5}, boundary='trim').sum()
               / At.coarsen({'y':5,'x':5}, boundary='trim').sum()
              )
        pres = xr.apply_ufunc(gsw.p_from_z, -SA.deptht, SA.nav_lat[2::5,2::5],
                              dask='parallelized', output_dtypes=[float,]
                             )
        N2, pN2 = gsw.Nsquared(s_b, t_b, pres, SA.nav_lat[2::5,2::5], axis=0)
        N2 = xr.DataArray(N2, dims=['depthw','y','x'], 
                          coords={'depthw':dsW.depthw[1:].data}
                         )
    #     del SA, CT

        z10 = 6
        sig0_b = xr.apply_ufunc(gsw.sigma0, s_b, t_b, 
                                dask='parallelized', output_dtypes=[float,]
                               )
        b_b = -g*sig0_b*1e-3

        nMLD_b = z10 + np.abs((sig0_b.isel(deptht=slice(z10,None))
                               - sig0_b.isel(deptht=z10)
                              ) - 0.03).argmin(dim='deptht').compute()
        MLD = (xr.ones_like(sig0_b) * sig0_b.deptht
              ).isel({'deptht':nMLD_b})
        tML = t_b.isel({'deptht':nMLD_b})
        sML = s_b.isel({'deptht':nMLD_b})
        bML = b_b.isel({'deptht':nMLD_b})
        del nMLD_b, sig0_b
#         if i == days[0,0]:
#             MLD_b = MLD
#         else:
#             MLD_b = xr.concat([MLD_b, MLD], 'time_counter')


#######################    
#         bb_ML = ((b_b * xr.DataArray(e3t.data[:len(b_b.deptht)],dims=['deptht'])
#                  ).where(b_b.deptht <= MLD).sum('deptht') 
#                  * MLD**-1)

        mu1 = 2*(-b_b.deptht) * MLD**-1 + 1
        mu1 = mu1.where(mu1.deptht <= MLD)
        mu = ((1.-mu1**2)*(1.+5/21*mu1**2)).chunk({'deptht':1})

        
        dbx = (b_b*e2tb).diff(dim='x', label='upper') * Aub[:,:-1]**-1
        dby = (b_b*e1tb).diff(dim='y', label='upper') * Avb[:-1]**-1
#         dbz = xr.DataArray((-b_b.diff(dim='deptht')
#                             * xr.DataArray(e3w[1:len(dsW.depthw)].data, dims=['deptht'])**-1).data, 
#                            dims=['deptht','y','x'],
#                            coords={'deptht':dsW.depthw[1:].data}
#                           ).chunk({'deptht':-1}).interp(deptht=dsT.deptht.data)
        dbx = (dbx.isel(x=slice(1,None))
               + dbx.isel(x=slice(None,-1)).data
              ) * .5
        dby = (dby.isel(y=slice(1,None)) 
               + dby.isel(y=slice(None,-1)).data
              ) * .5
        
#         dtdbx = (t_b*e2tb).diff(dim='x', label='upper') * (b_b*e2tb).diff(dim='x', label='upper')**-1
#         dtdby = (t_b*e1tb).diff(dim='y', label='upper') * (b_b*e1tb).diff(dim='y', label='upper')**-1
#         dtdbz = xr.DataArray((t_b.diff(dim='deptht')
#                               * b_b.diff(dim='deptht')**-1), 
#                              dims=['deptht','y','x'],
#                              coords={'deptht':dsW.depthw[1:].data}
#                             ).chunk({'deptht':-1}).interp(deptht=dsT.deptht.data)
#         dtx = (dtdbx.isel(x=slice(1,None))
#                + dtdbx.isel(x=slice(None,-1)).data
#               ) * .5
#         dty = (dtdby.isel(y=slice(1,None)) 
#                + dtdby.isel(y=slice(None,-1)).data
#               ) * .5
        dtx = (t_b*e2tb).diff(dim='x', label='upper') * Aub[:,:-1]**-1
        dty = (t_b*e1tb).diff(dim='y', label='upper') * Avb[:-1]**-1
        dtx = (dtx.isel(x=slice(1,None))
               + dtx.isel(x=slice(None,-1)).data
              ) * .5
        dty = (dty.isel(y=slice(1,None)) 
               + dty.isel(y=slice(None,-1)).data
              ) * .5
        
#         dsdbx = (s_b*e2tb).diff(dim='x', label='upper') * (b_b*e2tb).diff(dim='x', label='upper')**-1
#         dsdby = (s_b*e1tb).diff(dim='y', label='upper') * (b_b*e1tb).diff(dim='y', label='upper')**-1
#         dsdbz = xr.DataArray((s_b.diff(dim='deptht')
#                               * b_b.diff(dim='deptht')**-1), 
#                              dims=['deptht','y','x'],
#                              coords={'deptht':dsW.depthw[1:].data}
#                             ).chunk({'deptht':-1}).interp(deptht=dsT.deptht.data)
#         dsx = (dsdbx.isel(x=slice(1,None))
#                + dsdbx.isel(x=slice(None,-1)).data
#               ) * .5
#         dsy = (dsdby.isel(y=slice(1,None)) 
#                + dsdby.isel(y=slice(None,-1)).data
#               ) * .5
        dsx = (s_b*e2tb).diff(dim='x', label='upper') * Aub[:,:-1]**-1
        dsy = (s_b*e1tb).diff(dim='y', label='upper') * Avb[:-1]**-1
        dsx = (dsx.isel(x=slice(1,None))
               + dsx.isel(x=slice(None,-1)).data
              ) * .5
        dsy = (dsy.isel(y=slice(1,None)) 
               + dsy.isel(y=slice(None,-1)).data
              ) * .5
        
        
#         dtdb = (dtz.isel(y=slice(1,-1),x=slice(1,-1)) 
#                 + dty.isel(x=slice(1,-1)) 
#                 + dtx.isel(y=slice(1,-1)))
#         dsdb = (dsz.isel(y=slice(1,-1),x=slice(1,-1)) 
#                 + dsy.isel(x=slice(1,-1)) 
#                 + dsx.isel(y=slice(1,-1)))
#         dtdbH = (dtdb * xr.DataArray(e3t.data[:len(b_b.deptht)],dims=['deptht'])
#                 ).where(dtdb.deptht <= MLD.isel(y=slice(1,-1),x=slice(1,-1))).sum('deptht') 
#         dsdbH = (dsdb * xr.DataArray(e3t.data[:len(b_b.deptht)],dims=['deptht'])
#                 ).where(dsdb.deptht <= MLD.isel(y=slice(1,-1),x=slice(1,-1))).sum('deptht') 
#         dbx = (dbx * xr.DataArray(e3t.data[:len(b_b.deptht)],dims=['deptht'])
#               ).where(dbx.deptht <= MLD.isel(x=slice(1,-1))).sum('deptht') * MLD.isel(x=slice(1,-1))**-1
#         dby = (dby * xr.DataArray(e3t.data[:len(b_b.deptht)],dims=['deptht'])
#               ).where(dby.deptht <= MLD.isel(y=slice(1,-1))).sum('deptht') * MLD.isel(y=slice(1,-1))**-1
#         # print(dty, dby, dtx, dbx)    
        dbxH = ((dbx * xr.DataArray(e3t.data[:len(b_b.deptht)],dims=['deptht'])
                ).where(dbx.deptht <= MLD.isel(x=slice(1,-1))).sum('deptht') 
               )
        dbyH = ((dby * xr.DataArray(e3t.data[:len(b_b.deptht)],dims=['deptht'])
                ).where(dby.deptht <= MLD.isel(y=slice(1,-1))).sum('deptht') 
               )
        dtxH = ((dtx * xr.DataArray(e3t.data[:len(b_b.deptht)],dims=['deptht'])
                ).where(dtx.deptht <= MLD.isel(x=slice(1,-1))).sum('deptht') 
               )
        dtyH = ((dty * xr.DataArray(e3t.data[:len(b_b.deptht)],dims=['deptht'])
                ).where(dty.deptht <= MLD.isel(y=slice(1,-1))).sum('deptht') 
               )
        dsxH = ((dsx * xr.DataArray(e3t.data[:len(b_b.deptht)],dims=['deptht'])
                ).where(dsx.deptht <= MLD.isel(x=slice(1,-1))).sum('deptht') 
               )
        dsyH = ((dsy * xr.DataArray(e3t.data[:len(b_b.deptht)],dims=['deptht'])
                ).where(dsy.deptht <= MLD.isel(y=slice(1,-1))).sum('deptht') 
               )
        dtzH = (t_b.isel(deptht=0) - tML) 
        dszH = (s_b.isel(deptht=0) - sML) 
        dbzH = (b_b.isel(deptht=0) - bML)
        
        
        dtdbH = ((dtzH * dbzH**-1).isel(y=slice(1,-1),x=slice(1,-1))
                 + (dtyH * dbyH**-1).isel(x=slice(1,-1)) 
                 + (dtxH * dbxH**-1).isel(y=slice(1,-1)))
        dsdbH = ((dszH * dbzH**-1).isel(y=slice(1,-1),x=slice(1,-1))
                 + (dsyH * dbyH**-1).isel(x=slice(1,-1)) 
                 + (dsxH * dbxH**-1).isel(y=slice(1,-1)))
        gradB2 = dbxH.isel(y=slice(1,-1))**2 + dbyH.isel(x=slice(1,-1))**2
    #     del b_b, dbx, dby

        gradbb2H = (Ce 
                    * gradB2 
                    * np.abs(fb).isel(y=slice(1,-1),x=slice(1,-1))**-1
                   )

#         wsbsb_ML = wsbs_b.where(wsbs_b.depthw <= MLD)
#####################  
        if i == days[0,0]:
#             Omdt_bar = (gradbb2H * ((N2 * xr.DataArray(e3w[1:len(dsW.depthw)], dims='depthw',
#                                                        coords={'depthw':dsW.depthw[1:].data}
#                                                       )).sum('depthw')**-1
#                                     * (t_b.isel(deptht=z10)-tML) 
#                                    ).isel(y=slice(1,-1),x=slice(1,-1))
#                         * (mu.isel(y=slice(1,-1),x=slice(1,-1)) 
#                            * xr.DataArray(e3t[:len(dsT.deptht)], dims='deptht',
#                                           coords={'deptht':dsT.deptht.data}
#                                          )
#                           ).sum('deptht') * MLD.isel(y=slice(1,-1),x=slice(1,-1))**-1
#                        ).compute()
#             Omds_bar = (gradbb2H * ((N2 * xr.DataArray(e3w[1:len(dsW.depthw)], dims='depthw',
#                                                        coords={'depthw':dsW.depthw[1:].data}
#                                                       )).sum('depthw')**-1
#                                     * (s_b.isel(deptht=z10)-sML) 
#                                    ).isel(y=slice(1,-1),x=slice(1,-1))
#                         * (mu.isel(y=slice(1,-1),x=slice(1,-1)) 
#                            * xr.DataArray(e3t[:len(dsT.deptht)], dims='deptht',
#                                           coords={'deptht':dsT.deptht.data}
#                                          )
#                           ).sum('deptht') * MLD.isel(y=slice(1,-1),x=slice(1,-1))**-1
#                        ).compute()
            Omdt_bar = (gradbb2H 
#                         * ((N2 * xr.DataArray(e3w[1:len(dsW.depthw)], dims='depthw',
#                                               coords={'depthw':dsW.depthw[1:].data}
#                                              )).sum('depthw')**-1 
#                           ).isel(y=slice(1,-1),x=slice(1,-1))
                        * dtdbH
                        * (mu.isel(y=slice(1,-1),x=slice(1,-1)) 
                           * xr.DataArray(e3t[:len(dsT.deptht)], dims='deptht',
                                          coords={'deptht':dsT.deptht.data}
                                         )
                          ).sum('deptht') * MLD.isel(y=slice(1,-1),x=slice(1,-1))**-1
                       ).compute()
            Omds_bar = (gradbb2H 
#                         * ((N2 * xr.DataArray(e3w[1:len(dsW.depthw)], dims='depthw',
#                                               coords={'depthw':dsW.depthw[1:].data}
#                                              )).sum('depthw')**-1 
#                           ).isel(y=slice(1,-1),x=slice(1,-1))
                        * dsdbH
                        * (mu.isel(y=slice(1,-1),x=slice(1,-1)) 
                           * xr.DataArray(e3t[:len(dsT.deptht)], dims='deptht',
                                          coords={'deptht':dsT.deptht.data}
                                         )
                          ).sum('deptht') * MLD.isel(y=slice(1,-1),x=slice(1,-1))**-1
                       ).compute()
        else:
#             Omdt_bar = xr.concat([Omdt_bar, (gradbb2H * ((N2 * xr.DataArray(e3w[1:len(dsW.depthw)], dims='depthw',
#                                                                             coords={'depthw':dsW.depthw[1:].data}
#                                                                            )).sum('depthw')**-1
#                                                          * (t_b.isel(deptht=z10)-tML) 
#                                                         ).isel(y=slice(1,-1),x=slice(1,-1))
#                                              * (mu.isel(y=slice(1,-1),x=slice(1,-1)) 
#                                                 * xr.DataArray(e3t[:len(dsT.deptht)], dims='deptht',
#                                                                coords={'deptht':dsT.deptht.data}
#                                                               )
#                                                ).sum('deptht') * MLD.isel(y=slice(1,-1),x=slice(1,-1))**-1
#                                             ).compute()
#                                  ],
#                                  'time_counter')
#             Omds_bar = xr.concat([Omds_bar, (gradbb2H * ((N2 * xr.DataArray(e3w[1:len(dsW.depthw)], dims='depthw',
#                                                                             coords={'depthw':dsW.depthw[1:].data}
#                                                                            )).sum('depthw')**-1
#                                                          * (s_b.isel(deptht=z10)-sML) 
#                                                         ).isel(y=slice(1,-1),x=slice(1,-1))
#                                              * (mu.isel(y=slice(1,-1),x=slice(1,-1)) 
#                                                 * xr.DataArray(e3t[:len(dsT.deptht)], dims='deptht',
#                                                                coords={'deptht':dsT.deptht.data}
#                                                               )
#                                                ).sum('deptht') * MLD.isel(y=slice(1,-1),x=slice(1,-1))**-1
#                                             ).compute()
#                                  ],
#                                  'time_counter')
            Omdt_bar = xr.concat([Omdt_bar, (gradbb2H 
#                                              * ((N2 * xr.DataArray(e3w[1:len(dsW.depthw)], dims='depthw',
#                                                                    coords={'depthw':dsW.depthw[1:].data}
#                                                                   )).sum('depthw')**-1
#                                                ).isel(y=slice(1,-1),x=slice(1,-1))
                                             * dtdbH
                                             * (mu.isel(y=slice(1,-1),x=slice(1,-1)) 
                                                * xr.DataArray(e3t[:len(dsT.deptht)], dims='deptht',
                                                               coords={'deptht':dsT.deptht.data}
                                                              )
                                               ).sum('deptht') * MLD.isel(y=slice(1,-1),x=slice(1,-1))**-2
                                            ).compute()
                                 ],
                                 'time_counter')
            Omds_bar = xr.concat([Omds_bar, (gradbb2H 
#                                              * ((N2 * xr.DataArray(e3w[1:len(dsW.depthw)], dims='depthw',
#                                                                    coords={'depthw':dsW.depthw[1:].data}
#                                                                   )).sum('depthw')**-1
#                                                ).isel(y=slice(1,-1),x=slice(1,-1))
                                             * dsdbH
                                             * (mu.isel(y=slice(1,-1),x=slice(1,-1)) 
                                                * xr.DataArray(e3t[:len(dsT.deptht)], dims='deptht',
                                                               coords={'deptht':dsT.deptht.data}
                                                              )
                                               ).sum('deptht') * MLD.isel(y=slice(1,-1),x=slice(1,-1))**-2
                                            ).compute()
                                 ],
                                 'time_counter')

        del MLD, gradB2, gradbb2H
        
#         client.restart()
        print(str(l)+'-'+str(i))
    
#     wsbsb_bar.to_netcdf(op.join(scratch,'wsbs_bar_2009-%2d-%02d_%2d-%02d.nc' % (j,days[k,0],l,days[k,-1])))

dsave = Omdt_bar.to_dataset(name='Tflux')
dsave['Sflux'] = Omds_bar
dsave.to_netcdf(op.join(scratch,'GulfStream/Omd_t-s_bar-24h_%4d-%02d-%02d_%02d-%02d.nc' % (year,j,days[0,0],m,days[-1,-1])))
    
#     if k < len(dirs)-2:
#         del BFK_bar

4-11
4-12
4-13
4-14
4-15
4-16
4-17
4-18
4-19
4-20
4-21
4-22
4-23
4-24
4-25
4-26
4-27
4-28
4-29
4-30
5-1
5-2
5-3
5-4
5-5


In [ ]:
fig, (ax1,ax2) = plt.subplots(nrows=1, ncols=2, figsize=(20,7))
fig.set_tight_layout(True)
wsbsb_bar[0].plot(ax=ax1, vmin=-1e-7, vmax=1e-7, cmap='RdBu_r')
(BFK_bar[0]*.1).plot(ax=ax2, vmin=0, vmax=1e-7, cmap='Reds')

In [ ]:
MLD_b.mean('time_counter').plot(vmax=1.4e2, vmin=20, cmap='Blues')

In [ ]:
wsbsb_ML.chunk({'depthw':-1}).mean(['time_counter','depthw']).plot(figsize=(8,5), cmap='RdBu_r', vmin=-1e-7, vmax=1e-7)

In [ ]:
fig, ax = plt.subplots(figsize=(14,4))
wsbs_bar.plot(ax=ax, c='r', lw=3)
gradbb2H_bar.plot(ax=ax, c='k', ls='--', lw=2)

In [ ]:
gradbb2H.name = 'BFK'
wsbsb_ML.name = 'wsbs'
from matplotlib.gridspec import GridSpec

fig = plt.figure(figsize=(20,5))
# fig.set_tight_layout(True)
gs = GridSpec(1,5,figure=fig)
ax1 = fig.add_subplot(gs[0,:2])
ax2 = fig.add_subplot(gs[0,2:4])
ax3 = fig.add_subplot(gs[0,4])
wsbsb_ML[0,14,1:-1,1:-1].plot(ax=ax1, vmax=5e-8, cmap='coolwarm', rasterized=True)
(1e2*gradbb2H[15,0]).plot(ax=ax2, vmin=0., vmax=5e-8, cmap='Reds', rasterized=True)
xhist(1e2*gradbb2H[1:].transpose('time_counter','deptht','y','x').chunk({'time_counter':1}), 
      wsbsb_ML[:,:,1:-1,1:-1].chunk({'time_counter':1}),
      bins=[np.logspace(-11,-6,30), np.logspace(-11,-6,30)]).plot(ax=ax3, cmap='Blues')
ax3.plot(np.linspace(1e-11,1e-6), np.linspace(1e-11,1e-6), c='grey', ls='--')
ax3.set_xscale('log')
ax3.set_yscale('log')
ax3.set_ylim([1e-11,1e-6])
ax3.set_xlim([1e-11,1e-6])
ax3.set_xlabel(r"$\overline{w'b'}^{xy}$ [m$^2$ s$^{-3}$]", fontsize=14)
ax3.set_ylabel(r"$\frac{H_{\tt ML}^2|\nabla_{\tt H}\overline{b}^{xyz}|^2}{f}$ [m$^2$ s$^{-3}$]", 
               fontsize=14)
ax1.set_title(r"$\overline{w'b'}^{xy}$ [m$^2$ s$^{-3}$]", fontsize=15)
ax2.set_title(r"$H_{\tt ML}^2|\nabla_{\tt H}\overline{b}^{xyz}|^2 / f$ [m$^2$ s$^{-3}$]", 
             fontsize=15)
fig.suptitle(r"Nov. 2, 2009", fontsize=17, y=1.02)
# plt.savefig('/scratch/cnt0024/hmg2840/tuchida/Figs/Nov-2-2009_GulfStreamPatch.pdf')